# TODO :  change from sorting by resource type to profile

## Create FHIR R4 SearchParameter Resource

Create FHIR R4 SearchParameter Resource, Quick start text, and Searchparameter list using the python fhir client

Source data is in excel file

### Prerequisites:

- Python 3.7 or greater


### Import FHIRR4Client and other libraries

In [47]:
%config IPCompleter.greedy=True

In [48]:
from fhirclient.r4models.fhirabstractbase import FHIRValidationError
from fhirclient.r4models import searchparameter as SP
from fhirclient.r4models import capabilitystatement as CS
from fhirclient.r4models import bundle as B
import fhirclient.r4models.identifier as I
import fhirclient.r4models.coding as C
import fhirclient.r4models.codeableconcept as CC
import fhirclient.r4models.fhirdate as D
import fhirclient.r4models.extension as X
import fhirclient.r4models.contactdetail as CD
from json import dumps, loads, load
from requests import get, post, put
import os
from pathlib import Path
from csv import reader as csvreader
from IPython.display import display as Display, HTML, Markdown
from pprint import pprint
from collections import namedtuple
from pandas import *
from datetime import datetime
from jinja2 import Environment, FileSystemLoader, select_autoescape
import R4sp_summary_list as sp_map
from stringcase import snakecase, titlecase, pascalcase
from itertools import zip_longest
from openpyxl import load_workbook

####  Assign Global Variables


Here is where we assign all the global variables for this example such as the local paths for file input and output

##### Need to update:
- base_id
- paths
- canonical

In [49]:
#******************** Need to update *************************************************
fhir_base_url = 'http://hl7.org/fhir/'
base_id = "US-Core"
canon_base = "http://hl7.org/fhir/us/core/"
ig_folder = 'US-Core'
publisher = 'HL7 International - Structured Documents Work Group'
publisher_endpoint = dict(
                        system = 'url',
                        value = 'http://www.hl7.org/Special/committees/structure/index.cfm'
                        ) 
#ig_source_path = "//ERICS-AIR-2/ehaas/Documents/FHIR/US-Core-R4/source/"
ig_source_path = ''
#ig_pages_path= '/Users/ehaas/Documents/FHIR/US-Core-R4/source/pages/_includes'
#***********************************************************************************

env = Environment(
    loader=FileSystemLoader(searchpath = ''),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )

md_template = ['quick_start.j2', 'sp_list_page.j2', 'cs_search_documentation.j2','sp_narrative.j2']


fhir_term_server = 'http://test.fhir.org/r3'

# profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient



none_list = ['', ' ', 'none', 'n/a', 'N/A', 'N', 'False', 'FALSE']
sep_list = (',', ';', ' ', ', ', '; ')



### validate

In [50]:
# *********************** validate Resource as Dict ********************************

def validate(r):

    fhir_test_server = 'http://test.fhir.org/r4'

    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }
    
    params = {
    }
    
    r = post(f'{fhir_test_server}/{r["resourceType"]}/$validate', params = params, headers = headers, data = dumps(r))
    # return r.status_code
    # view  output
    # return (r.json()["text"]["div"])
    return r

### Write to File

In [51]:
def write_file(f_name, data): # write file
    path = Path.cwd() / 'search_md' / f_name
    #path = pathlib.Path(r'/Users/ehaas/Documents/FHIR/US-Core-R4/source/pages/_includes')
    path.write_text(data)

write_file('foo.md', '## foo')

### Get Search Parameter input data

In [52]:
#in_path = '/Users/ehaas/Documents/FHIR/pyfhir/test/'
in_path =''

in_file ="uscore-server"

xls = ExcelFile(f'{in_path}{in_file}.xlsx')
df = read_excel(xls,'sps',na_filter = False)
df_combos = read_excel(xls,'sp_combos',na_filter = False)

df_combos

,base,combo,combo_conf,types,description,example,imp_note
0,Encounter,"class,date",SHOULD,"date,token",,,
1,Encounter,"class,date,patient",SHOULD,"date,reference,token",,,
2,Encounter,"class,date,patient,type",SHOULD,"date,reference,token",,,
3,Encounter,"class,date,type",SHOULD,"date,token",,,
4,Encounter,"class,patient",SHOULD,"reference,token",,,
5,Encounter,"class,patient,status",SHOULD,"reference,token",,,
6,Encounter,"class,patient,status,type",SHOULD,"reference,token",,,
7,Encounter,"class,patient,type",SHOULD,"reference,token",,,
8,Encounter,"class,status",SHOULD,token,,,
9,Encounter,"class,status,type",SHOULD,token,,,


In [53]:
data = [i for i in df.itertuples(index=True)]
r_type =  {d.base for d in data if '!' not in d.base}
combo_data = [i for i in df_combos.itertuples(index=True)]

for d in data:
    print(f'Resource = {d.base}, Search Parameter = {d.code}, Exists = {d.exists}')
for c in combo_data:
    print(f'Resource = {c.base}, Combo Search Parameter = {c.combo}')

Resource = Questionnaire, Search Parameter = _id, Exists = Y
Resource = Questionnaire, Search Parameter = url, Exists = Y
Resource = Questionnaire, Search Parameter = status, Exists = Y
Resource = Questionnaire, Search Parameter = title, Exists = Y
Resource = Questionnaire, Search Parameter = publisher, Exists = Y
Resource = Questionnaire, Search Parameter = version, Exists = Y
Resource = Questionnaire, Search Parameter = context-type-value, Exists = N
Resource = AllergyIntolerance, Search Parameter = _id, Exists = Y
Resource = Patient, Search Parameter = _id, Exists = Y
Resource = Patient, Search Parameter = identifier, Exists = Y
Resource = Patient, Search Parameter = name, Exists = Y
Resource = Patient, Search Parameter = birthdate, Exists = Y
Resource = Patient, Search Parameter = gender, Exists = Y
Resource = Patient, Search Parameter = family, Exists = Y
Resource = Patient, Search Parameter = given, Exists = Y
Resource = !Patient, Search Parameter = address, Exists = Y
Resource =

### update core SP with additional capabiliities


- Get definitions bundle and convert to python object for ease of notation
- use sp_map to map to Type + parameter
- If need to update SP Extract the SP based on the excel file

### load SP Mapping dictionary

output shows a single SP entry for a sanity check

In [54]:
p = Path('C:/Users/Eric/Documents/HL7/FHIR/BUILD_EDIT_FILES/R4_Definitions/search-parameters.json')

b = B.Bundle(loads(p.read_text()), strict = False)
sp = b.entry[0]
print(sp.fullUrl)
pprint([i for i in sp_map.sp_list2 if i[0]=="Observation" and i[1] == 'code'])
sp.as_json()

http://hl7.org/fhir/SearchParameter/DomainResource-text
[('Observation', 'code', 'http://hl7.org/fhir/SearchParameter/clinical-code')]


{'fullUrl': 'http://hl7.org/fhir/SearchParameter/DomainResource-text',
 'resource': {'id': 'DomainResource-text',
  'extension': [{'url': 'http://hl7.org/fhir/StructureDefinition/structuredefinition-standards-status',
    'valueCode': 'trial-use'}],
  'base': ['DomainResource'],
  'code': '_text',
  'contact': [{'telecom': [{'system': 'url', 'value': 'http://hl7.org/fhir'}]},
   {'telecom': [{'system': 'url',
      'value': 'http://www.hl7.org/Special/committees/fiwg/index.cfm'}]}],
  'date': '2019-04-18T15:23:58+00:00',
  'description': 'Search on the narrative of the resource',
  'experimental': False,
  'modifier': ['missing', 'exact', 'contains'],
  'multipleAnd': True,
  'multipleOr': True,
  'name': '_text',
  'publisher': 'Health Level Seven International (FHIR Infrastructure)',
  'status': 'draft',
  'type': 'string',
  'url': 'http://hl7.org/fhir/SearchParameter/DomainResource-text',
  'version': '4.1.0',
  'xpathUsage': 'normal',
  'resourceType': 'SearchParameter'}}

## create updated SPs
sp optional 'modifier' elements are listed as comma separated list of shalls and shoulds for each:
- multipleOr
- multipleOr_conf
- multipleAnd
- multipleAnd_conf
- shall_modifier
- should_modifier
- shall_comparator
- should_comparator
- shall_chain

if not spedified then conformance is MAY 

*note that it starts out using the FHIRClient models but the switches to a dict structure to add the FHIR primitive type extensions.*

In [55]:
def sp_expectation(conf=None):
    if not conf:
        conf = "MAY"
 
    x = X.Extension(dict(
    url = f'http://hl7.org/fhir/StructureDefinition/capabilitystatement-expectation',
    valueCode = conf
    ))
    x_dict = dict(
    extension = [x.as_json()]
    )
 
    
    return x_dict

prefix = 'us-core-'

sp_list=[]
for d in data:
    print(f'========={d.base},{d.code}==============')
    for i in b.entry:
        sp = i.resource
        if '!' not in d.base and d.update =='Y' and d.base in sp.base and d.code == sp.code: #update sp
            # print(dumps(i.resource.as_json(),indent = 3))
            # change id and url, publisher, and contact, draft etc
            sp.id = f'{base_id.lower()}-{d.base.lower()}-{d.code}'
            sp.extension = []
            sp.derivedFrom =sp.url
            sp.url = f'{canon_base}SearchParameter/{sp.id}'
            sp.publisher = publisher
            sp.contact = [CD.ContactDetail( {"telecom" : [ publisher_endpoint ] })]
            sp.date = D.FHIRDate(f'{datetime.utcnow().isoformat()}Z')
            sp.name = pascalcase(sp.id.replace('-','_'))
            sp.name = sp.name.replace('UsCore','USCore')
            sp.status = 'active'
            sp.base = [d.base]
            try:
                sp.expression = [i for i in sp.expression.split('|') if i.startswith(f'{d.base}.')][0]
            except IndexError:
                sp.expression = [i for i in sp.expression.split(' | ') if i.startswith(f'{d.base}.')][0]
            try:
                sp.xpath = [i for i in sp.xpath.split('|') if i.startswith(f'f:{d.base}/')][0]
            except IndexError:
                sp.xpath = [i for i in sp.xpath.split(' | ') if i.startswith(f'f:{d.base}/')][0]
            except AttributeError:
                pass
            try:   
                sp.description = [i for i in sp.description.split('\r\n* ') if i.startswith(f'[{d.base}]')][0]
                sp.description = sp.description.split(': ')[1:]
                sp.description = ''.join(sp.description)
            except IndexError:   
                print(sp.expression)
      
            #convert to dict since model can't handle primitive extensions
            sp_dict = sp.as_json()

            sp_dict['multipleOr'] = False if d.multipleOr in none_list else True
            sp_dict['_multipleOr'] = sp_expectation(d.multipleOr_conf)
            
            sp_dict['multipleAnd'] = False if d.multipleAnd in none_list else True
            sp_dict['_multipleAnd'] = sp_expectation(d.multipleAnd_conf)

            try:
                sp_dict['_modifier'] = []
                for m in sp_dict['modifier']: # list all modifiers in sp and assign an expectation.
                    if d.shall_modifier not in none_list and m in d.shall_modifier.split(','):
                       sp_dict['_modifier'].append(sp_expectation('SHALL'))
                    elif  d.should_modifier not in none_list and m in d.should_modifier.split(','):
                        sp_dict['_modifier'].append(sp_expectation('SHOULD'))               
                    else:
                        sp_dict['_modifier'].append(sp_expectation('MAY'))
            except KeyError:
                del(sp_dict['_modifier'])

            try:
                sp_dict['_comparator'] = []
                for m in sp_dict['comparator']: # list all comparators in sp and assign an expectation.
                   if d.shall_comparator not in none_list and m in d.shall_comparator.split(','):
                       sp_dict['_comparator'].append(sp_expectation('SHALL'))
                   elif  d.should_comparator not in none_list and m in d.should_comparator.split(','):
                        sp_dict['_comparator'].append(sp_expectation('SHOULD'))               
                   else:
                        sp_dict['_comparator'].append(sp_expectation('MAY'))
            except KeyError:
                del(sp_dict['_comparator'])

            if d.shall_chain not in none_list:
               sp_dict['chain'] = d.shall_chain.split(',')
               sp_dict['_chain'] = [sp_expectation('SHALL') for c in d.shall_chain.split(',')]

            if d.should_chain not in none_list:
               sp_dict['chain'] = d.should_chain.split(',')
               sp_dict['_chain'] = [sp_expectation('SHALL') for c in d.should_chain.split(',')]

            print(f'======================= SP = {sp_dict["id"]} =====================')
            #print(dumps(sp_dict,indent=4))
                  
            #================ add narrative =======================
            template = env.get_template(md_template[3])   
            sp_text = template.render(sp=sp_dict)
            #print(sp_text)     
            display(HTML(sp_text))
                  
            sp_dict['text'] = {}
            sp_dict['text']['status'] = "generated"
            sp_dict['text']['div'] = sp_text      
                  
            # ================ save files as resource ======================
            # save in temp folder
            path = Path.cwd() / 'sp_json' / f'searchparameter-{sp_dict["id"]}.json'
            # save in ig_source folder
            #path = Path.cwd() / ig_source_path / 'resources' / f'searchparameter-{sp_dict["id"]}.json'
 
            path.write_text(dumps(sp_dict,indent=4))
            sp_list.append(sp_dict)

=========Questionnaire,_id==============
=========Questionnaire,url==============
Questionnaire.url
======================= SP = us-core-questionnaire-url =====================


=========Questionnaire,status==============
Questionnaire.status
======================= SP = us-core-questionnaire-status =====================


=========Questionnaire,title==============
Questionnaire.title
======================= SP = us-core-questionnaire-title =====================


=========Questionnaire,publisher==============
Questionnaire.publisher
======================= SP = us-core-questionnaire-publisher =====================


=========Questionnaire,version==============
Questionnaire.version
======================= SP = us-core-questionnaire-version =====================


...

=========Encounter,date==============
======================= SP = us-core-encounter-date =====================


=========Encounter,identifier==============
======================= SP = us-core-encounter-identifier =====================


=========Encounter,status==============
Encounter.status
======================= SP = us-core-encounter-status =====================


=========Encounter,class==============
Encounter.class
======================= SP = us-core-encounter-class =====================


=========Encounter,type==============
======================= SP = us-core-encounter-type =====================


### Validate

In [56]:
for i in sp_list:
    print(f'Validating {i["id"]}...........')
    r = validate(i)
    display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))               

Validating us-core-questionnaire-url...........


Validating us-core-questionnaire-status...........


Validating us-core-questionnaire-title...........


Validating us-core-questionnaire-publisher...........


Validating us-core-questionnaire-version...........


...

Validating us-core-encounter-date...........


Validating us-core-encounter-identifier...........


Validating us-core-encounter-status...........


Validating us-core-encounter-class...........


Validating us-core-encounter-type...........


### Create Ad Hoc Structure for listing all the required combination Search Parameters

- uses Spreadsheet
- uses set theory and math to enumarate combinations
- uses exclude list to remove prohibited combination
- uses least strong Conformance of members of combot to determine conformance


create a Dict with key = Resource_type and value equal to a sorted list of tuples with combonent parameter list, conformance  e.g. {Type:[(p_combo-1,c1),(p_combo2,c2...]}

add new sheet to spreadsheet for manual review

In [57]:
def get_conf_str(combo, r_type):
    conf_str = ''
    for k in df.itertuples(index=True):
        if k.base == r_type and k.code in combo:
            if  k.base_conf == 'MAY' or k.base_conf in none_list:
                conf_str = 'MAY'
                break
            elif k.base_conf == 'SHOULD':
                conf_str = 'SHOULD' 
            elif k.base_conf == 'SHALL' and conf_str not in ['SHALL','MAY']:
                conf_str ='SHALL' 
    return conf_str

def get_type_list(combo, r_type):
    t = {i.type for i in df.itertuples(index=True) if i.code in combo and i.base == r_type}
    print(t)
    return list(t)


def get_combos(pairs,c_list):
    a_list =[]
    for i in c_list:
        for j in c_list:
            #print(f'i={i} j = {j}, i&j= {i&j} i^j= {i^j}')
            if i & j and i != j:
                #print(f'i={i}, j = {j}, i&j= {i&j},i^j= {i^j} i|j = {i|j}')
                if i^j in pairs:
                    
                    if i|j not in a_list + c_list:
                        a_list.append(i|j)
    return a_list

          

combo_list = []
combo_dict = {}
for base in r_type:
    print(base)
    pairs = []
    sp_len = len([d for d in data if d.base== base])
          
    for d in data:
        if d.base == base and d.combo_pairs not in none_list: 
            pairs.extend([{d.code, p} for p in d.combo_pairs.split(',') if p != d.code])  # get allowed pairs
            #print(f'pairs = {pairs}')
          
    combo_list = pairs
    for j in range(sp_len-1):
         combo_list = combo_list + get_combos(pairs,combo_list)
          
    # convert to sorted tuples
    combo_list = [sorted(tuple(i)) for i in combo_list]
    combo_list = sorted(combo_list)
          
    try: #subtract forbidden combos
        f_combos = []
        df_r = read_excel(xls,'resources',na_filter = False)
        for r in df_r.itertuples(index=True):
            if  r.type == base:
                f_combos = r.forbidden_s_combos.split(',')
                #print(f_combos)
                f_combos = [sorted(i.split('|')) for i in f_combos]
                #print(f_combos)
        combo_list = [i for i in combo_list if sorted(i) not in f_combos]
    except AttributeError:
        print('forbidden_s_combos is missing')
          
    print(f'{len(combo_list)} combos = {combo_list}')
    #combo_list.extend(combo_list)
    combo_dict[base]=[(c, get_conf_str(combo = c, r_type = base), get_type_list(combo = c, r_type = base)) for c in combo_list]
# print(dumps(combo_dict, indent = 4))

AllergyIntolerance
0 combos = []
Encounter
18 combos = [['class', 'date'], ['class', 'date', 'patient'], ['class', 'date', 'patient', 'type'], ['class', 'date', 'type'], ['class', 'patient'], ['class', 'patient', 'status'], ['class', 'patient', 'status', 'type'], ['class', 'patient', 'type'], ['class', 'status'], ['class', 'status', 'type'], ['class', 'type'], ['date', 'patient'], ['date', 'patient', 'type'], ['date', 'type'], ['patient', 'status'], ['patient', 'status', 'type'], ['patient', 'type'], ['status', 'type']]
{'date', 'token'}
{'date', 'reference', 'token'}
{'date', 'reference', 'token'}
{'date', 'token'}
{'reference', 'token'}
{'reference', 'token'}
{'reference', 'token'}
{'reference', 'token'}
{'token'}
{'token'}
{'token'}
{'date', 'reference'}
{'date', 'reference', 'token'}
{'date', 'token'}
{'reference', 'token'}
{'reference', 'token'}
{'reference', 'token'}
{'token'}
Questionnaire
9 combos = [['context-type-value', 'publisher'], ['context-type-value', 'publisher', 'stat

### Convert combo_dict to dataframe and add to spreadsheet for review

In [58]:
rows=[]
for k,v in combo_dict.items():
    for i in v:
        rows.append([k, *i])
        
generated_combos= DataFrame(rows, columns = ['Base Type',"Combo",'Conformance','Parameter Types',] )

book = load_workbook(f'{in_path}{in_file}.xlsx')
std = book['generated_combos']  # remove sheet first if  exists
book.remove(std)
#update information
with ExcelWriter(f'{in_path}{in_file}.xlsx', engine = 'openpyxl') as writer:
    writer.book = book
    
    generated_combos.to_excel(writer, sheet_name = 'generated_combos')

### Create Markdown Text for Search.

- Using Jinja2 Template create markdown file for search section in profiles profiles
- use spreadhsheet as input data  ( could use the sp too )
- get combos from spreadsheet
  - update parameter types if needed
  

In [59]:

search_type = dict(
    token = '{[system]}|[code]',
    id = '[id]',
    reference = '[reference]',
    string = '[string]',
    uri = '[uri]',
    date = '[date]',
    )

template = env.get_template(md_template[0])

print(r_type)   
for t in r_type:  # preprocess the for jinja templates
    #t = 'AllergyIntolerance'

    sp = [d for d in data if d.base == t]
    mods= {}
    for s in sp:
            l1=s.shall_modifier.split(',') if s.shall_modifier else []
            l2=s.should_modifier.split(',') if s.should_modifier else []
            l3=s.shall_comparator.split(',') if s.shall_comparator else []
            l4=s.should_comparator.split(',') if s.should_comparator else []
            l5=s.shall_chain.split(',') if s.shall_chain else []
            l6=s.should_chain.split(',') if s.should_chain else []
            mods[s.code] = (l2+l2,l3+l4,l5+l6)
    
    #print(mods)
    print(combo_dict[t])
    shalls = "SHALL" in [i.base_conf for i in sp if i.base == t] + combo_dict[t]  # TODO need to search both singles and combos
    shoulds = "SHOULD" in [i.base_conf for i in sp if i.base == t] + combo_dict[t]
    #print(t)
    
    search_md = template.render(
                    r_type=t,
                    sp=sp,
                    search_type=search_type,
                    combos=combo_dict[t],
                    shalls=shalls,
                    shoulds=shoulds,
                    mods = mods,
                    )

    
    display(Markdown(search_md))
    # save in temp folder
    path = Path.cwd() / 'search_md' / f'us-core-{t.lower()}-search.md'
    #path = pathlib.Path(r'/Users/ehaas/Documents/FHIR/US-Core-R4/source/pages/_includes')
    path.write_text(search_md)

{'AllergyIntolerance', 'Encounter', 'Questionnaire', 'Patient'}
[]



#### Mandatory Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHALL be supported.  the  modifiers, comparators and chained parameters that are listed as optional SHOULD be supported.:


1. **SHALL** support searching for all allergies for a patient using the **`_id`** search parameter:

  `GET [base]/AllergyIntolerance[id]`

  Example: GET [base]/AllergyIntolerance?patient=1137192

  *Implementation Notes:* Fetches a bundle of all AllergyIntolerance resources for the specified patient (how to search by the [logical id] of the resource)





{% include link-list.md %}

[(['class', 'date'], 'SHOULD', ['date', 'token']), (['class', 'date', 'patient'], 'SHOULD', ['date', 'reference', 'token']), (['class', 'date', 'patient', 'type'], 'SHOULD', ['date', 'reference', 'token']), (['class', 'date', 'type'], 'SHOULD', ['date', 'token']), (['class', 'patient'], 'SHOULD', ['reference', 'token']), (['class', 'patient', 'status'], 'SHOULD', ['reference', 'token']), (['class', 'patient', 'status', 'type'], 'SHOULD', ['reference', 'token']), (['class', 'patient', 'type'], 'SHOULD', ['reference', 'token']), (['class', 'status'], 'SHOULD', ['token']), (['class', 'status', 'type'], 'SHOULD', ['token']), (['class', 'type'], 'SHOULD', ['token']), (['date', 'patient'], 'SHALL', ['date', 'reference']), (['date', 'patient', 'type'], 'SHOULD', ['date', 'reference', 'token']), (['date', 'type'], 'SHOULD', ['date', 'token']), (['patient', 'status'], 'SHOULD', ['reference', 'token']), (['patient', 'status', 'type'], 'SHOULD', ['reference', 'token']), (['patient', 'type'], 'SHO


#### Mandatory Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHALL be supported.  the  modifiers, comparators and chained parameters that are listed as optional SHOULD be supported.:


1. **SHALL** support fetching an Encounter using the **`_id`** search parameter:

  `GET [base]/Encounter[id]`

  Example: GET [base]/Encounter/12354

  *Implementation Notes:*  (how to search by the [logical id] of the resource)

1. **SHALL** support searching for all encounters for a patient using the **`patient`** search parameter:

  `GET [base]/Encounter?patient=[reference]`

  Example: GET [base]/Encounter?patient=1137192

  *Implementation Notes:* Fetches a bundle of all Encounter resources for the specified patient (how to search by [reference])

1. **SHALL** support searching for all encounters by date using the **`date`** search parameter:
  - including support for these comparators: `gt, lt, ge, le`

  `GET [base]/Encounter?date={gt|lt|ge|le}[date]`

  Example: See combination searches below

  *Implementation Notes:* Search based on date and patient parameter (how to search by [date])

1. **SHALL**  using the combination of the  **`date and patient`** search parameters:

  `GET [base]/Encounter?date={gt|lt|ge|le}[date]&patient=[reference]`

  Example: 

  *Implementation Notes:*  (how to search by [date] and [reference])



#### Optional Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHOULD be supported.

1. **SHOULD** support searching using the **`identifier`** search parameter:

  `GET [base]/Encounter?identifier={[system]}|[code]`

  Example: GET [base]/Encounter?identifier=http://hospital.smarthealthit.org|1032702

  *Implementation Notes:*  (how to search by [token])

1. **SHOULD** support searching using the **`status`** search parameter:

  `GET [base]/Encounter?status={[system]}|[code]`

  Example: GET [base]/Encounter?status=completed

  *Implementation Notes:* Fetches a bundle of all Encounter resources for the specified patient (how to search by [token])

1. **SHOULD** support searching using the **`class`** search parameter:

  `GET [base]/Encounter?class={[system]}|[code]`

  Example: GET [base]/Encounter?status=http://terminology.hl7.org/CodeSystem/v3-ActCode|AMB

  *Implementation Notes:* Fetches a bundle of all Encounter resources for the specified patient (how to search by [token])

1. **SHOULD** support searching using the **`type`** search parameter:

  `GET [base]/Encounter?type={[system]}|[code]`

  Example: GET [base]/Encounter?http://www.ama-assn.org/go/cpt|99201

  *Implementation Notes:* Fetches a bundle of all Encounter resources for the specified patient (how to search by [token])

1. **SHOULD** support searching using the combination of the  **`class and date`** search parameters:

  `GET [base]/Encounter?class=[token]&date={gt|lt|ge|le}[date]`

  Example: 

  *Implementation Notes:* . (how to search by [date] and [token])

1. **SHOULD** support searching using the combination of the  **`class and date and patient`** search parameters:

  `GET [base]/Encounter?class=[token]&date={gt|lt|ge|le}[date]&patient=[reference]`

  Example: 

  *Implementation Notes:* . (how to search by [date] and [reference] and [token])

1. **SHOULD** support searching using the combination of the  **`class and date and patient and type`** search parameters:

  `GET [base]/Encounter?class=[token]&date={gt|lt|ge|le}[date]&patient=[reference]&type=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [date] and [reference] and [token])

1. **SHOULD** support searching using the combination of the  **`class and date and type`** search parameters:

  `GET [base]/Encounter?class=[token]&date={gt|lt|ge|le}[date]&type=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [date] and [token])

1. **SHOULD** support searching using the combination of the  **`class and patient`** search parameters:

  `GET [base]/Encounter?class=[token]&patient=[reference]`

  Example: 

  *Implementation Notes:* . (how to search by [reference] and [token])

1. **SHOULD** support searching using the combination of the  **`class and patient and status`** search parameters:

  `GET [base]/Encounter?class=[token]&patient=[reference]&status=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [reference] and [token])

1. **SHOULD** support searching using the combination of the  **`class and patient and status and type`** search parameters:

  `GET [base]/Encounter?class=[token]&patient=[reference]&status=[token]&type=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [reference] and [token])

1. **SHOULD** support searching using the combination of the  **`class and patient and type`** search parameters:

  `GET [base]/Encounter?class=[token]&patient=[reference]&type=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [reference] and [token])

1. **SHOULD** support searching using the combination of the  **`class and status`** search parameters:

  `GET [base]/Encounter?class=[token]&status=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [token])

1. **SHOULD** support searching using the combination of the  **`class and status and type`** search parameters:

  `GET [base]/Encounter?class=[token]&status=[token]&type=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [token])

1. **SHOULD** support searching using the combination of the  **`class and type`** search parameters:

  `GET [base]/Encounter?class=[token]&type=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [token])

1. **SHOULD** support searching using the combination of the  **`date and patient and type`** search parameters:

  `GET [base]/Encounter?date={gt|lt|ge|le}[date]&patient=[reference]&type=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [date] and [reference] and [token])

1. **SHOULD** support searching using the combination of the  **`date and type`** search parameters:

  `GET [base]/Encounter?date={gt|lt|ge|le}[date]&type=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [date] and [token])

1. **SHOULD** support searching using the combination of the  **`patient and status`** search parameters:

  `GET [base]/Encounter?patient=[reference]&status=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [reference] and [token])

1. **SHOULD** support searching using the combination of the  **`patient and status and type`** search parameters:

  `GET [base]/Encounter?patient=[reference]&status=[token]&type=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [reference] and [token])

1. **SHOULD** support searching using the combination of the  **`patient and type`** search parameters:

  `GET [base]/Encounter?patient=[reference]&type=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [reference] and [token])

1. **SHOULD** support searching using the combination of the  **`status and type`** search parameters:

  `GET [base]/Encounter?status=[token]&type=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [token])


{% include link-list.md %}

[(['context-type-value', 'publisher'], 'SHOULD', ['composite', 'string']), (['context-type-value', 'publisher', 'status'], 'SHOULD', ['composite', 'string', 'token']), (['context-type-value', 'status'], 'SHOULD', ['composite', 'token']), (['publisher', 'status'], 'SHALL', ['string', 'token']), (['publisher', 'status', 'version'], 'SHOULD', ['string', 'token']), (['publisher', 'version'], 'SHOULD', ['string', 'token']), (['status', 'title', 'version'], 'SHOULD', ['string', 'token']), (['status', 'version'], 'SHOULD', ['token']), (['title', 'version'], 'SHOULD', ['string', 'token'])]



#### Mandatory Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHALL be supported.  the  modifiers, comparators and chained parameters that are listed as optional SHOULD be supported.:


1. **SHALL** support searching using the **`_id`** search parameter:

  `GET [base]/Questionnaire[id]`

  *Implementation Notes:* example for testing (how to search by the [logical id] of the resource)

1. **SHALL** support searching using the **`url`** search parameter:

  `GET [base]/Questionnaire?url=[uri]`

  *Implementation Notes:* example for testing (how to search by [uri])

1. **SHALL** support searching using the **`status`** search parameter:

  `GET [base]/Questionnaire?status={[system]}|[code]`

  *Implementation Notes:* example for testing (how to search by [token])

1. **SHALL** support searching using the **`title`** search parameter:
  - including optional support these modifiers: `foo, bar2`
  - including support for these comparators: `contains, foo, bar`
  - including support for these chained parameters: `contains, foo, bar`
    - and optional support these chained parameter: `foo, bar2`

  `GET [base]/Questionnaire?title{:foo|bar2|foo|bar2}={contains|foo|bar}[string]`

  *Implementation Notes:* example for testing (how to search by [string])

1. **SHALL** support searching using the **`publisher`** search parameter:
  - including support for these modifiers: `contains`

  `GET [base]/Questionnaire?publisher=[string]`

  *Implementation Notes:* example for testing (how to search by [string])

1. **SHALL**  using the combination of the  **`publisher and status`** search parameters:

  `GET [base]/Questionnaire?publisher=[string]&status=[token]`

  Example: 

  *Implementation Notes:*  (how to search by [string] and [token])



#### Optional Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHOULD be supported.

1. **SHOULD** support searching using the **`version`** search parameter:

  `GET [base]/Questionnaire?version={[system]}|[code]`

  *Implementation Notes:* example for testing (how to search by [token])

1. **SHOULD** support searching using the **`context-type-value`** search parameter:

  `GET [base]/Questionnaire?context-type-value=`

  *Implementation Notes:* example for testing (how to search by [composite])

1. **SHOULD** support searching using the combination of the  **`context-type-value and publisher`** search parameters:

  `GET [base]/Questionnaire?context-type-value=[composite]&publisher=[string]`

  Example: 

  *Implementation Notes:* . (how to search by [composite] and [string])

1. **SHOULD** support searching using the combination of the  **`context-type-value and publisher and status`** search parameters:

  `GET [base]/Questionnaire?context-type-value=[composite]&publisher=[string]&status=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [composite] and [string] and [token])

1. **SHOULD** support searching using the combination of the  **`context-type-value and status`** search parameters:

  `GET [base]/Questionnaire?context-type-value=[composite]&status=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [composite] and [token])

1. **SHOULD** support searching using the combination of the  **`publisher and status and version`** search parameters:

  `GET [base]/Questionnaire?publisher=[string]&status=[token]&version=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [string] and [token])

1. **SHOULD** support searching using the combination of the  **`publisher and version`** search parameters:

  `GET [base]/Questionnaire?publisher=[string]&version=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [string] and [token])

1. **SHOULD** support searching using the combination of the  **`status and title and version`** search parameters:

  `GET [base]/Questionnaire?status=[token]&title{:foo|bar2|foo|bar2}={contains|foo|bar}[string]&version=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [string] and [token])

1. **SHOULD** support searching using the combination of the  **`status and version`** search parameters:

  `GET [base]/Questionnaire?status=[token]&version=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [token])

1. **SHOULD** support searching using the combination of the  **`title and version`** search parameters:

  `GET [base]/Questionnaire?title{:foo|bar2|foo|bar2}={contains|foo|bar}[string]&version=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [string] and [token])


{% include link-list.md %}

[(['birthdate', 'family'], 'SHOULD', ['date', 'string']), (['birthdate', 'name'], 'SHALL', ['date', 'string']), (['family', 'gender'], 'SHOULD', ['string', 'token']), (['gender', 'name'], 'SHALL', ['string', 'token'])]



#### Mandatory Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHALL be supported.  the  modifiers, comparators and chained parameters that are listed as optional SHOULD be supported.:


1. **SHALL** support fetching a Patient using the **`_id`** search parameter:

  `GET [base]/Patient[id]`

  Example: GET [base]/Patient/1032702

  *Implementation Notes:*  (how to search by the [logical id] of the resource)

1. **SHALL** support searching a Patient by an identifier such as a MPI using the **`identifier`** search parameter:

  `GET [base]/Patient?identifier={[system]}|[code]`

  Example: GET [base]/Patient?identifier=http://hospital.smarthealthit.org|1032702

  *Implementation Notes:*  (how to search by [token])

1. **SHALL** support searching using the **`name`** search parameter:

  `GET [base]/Patient?name=[string]`

  Example: See combination searches below

  *Implementation Notes:* Search based on at least name and another patient element  (how to search by [string])

1. **SHALL**  using the combination of the  **`birthdate and name`** search parameters:

  `GET [base]/Patient?birthdate=[date]&name=[string]`

  Example: 

  *Implementation Notes:*  (how to search by [date] and [string])

1. **SHALL**  using the combination of the  **`gender and name`** search parameters:

  `GET [base]/Patient?gender=[token]&name=[string]`

  Example: 

  *Implementation Notes:*  (how to search by [string] and [token])



#### Optional Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHOULD be supported.

1. **SHOULD** support searching using the combination of the  **`birthdate and family`** search parameters:

  `GET [base]/Patient?birthdate=[date]&family=[string]`

  Example: 

  *Implementation Notes:* . (how to search by [date] and [string])

1. **SHOULD** support searching using the combination of the  **`family and gender`** search parameters:

  `GET [base]/Patient?family=[string]&gender=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [string] and [token])


{% include link-list.md %}

### Create Markdown Text for SearchParameters Page

- Using Jinja2 Template create markdown file for searchparameters page
- Use spreadsheet as source

In [60]:
for d in data:
    if '!' in d.base:
        print(d.base,True)

# get id for each row in spreadsheet:
d_tup = []
for d in data:
    if '!' not in d.base:
        sp_id= [sp['id'] for sp in sp_list if d.base in sp['base'] and sp['code'] in [d.code,'id'] ]
        d_tup.append((sp_id[0] if sp_id else None,d))

template = env.get_template(md_template[1])   

searchparameters_md = template.render(sp_list=d_tup,r_list={d.base for d in data if '!' not in d.base })

display(Markdown(searchparameters_md))
# save in temp folder
path = Path.cwd() / 'sp_page_md' / 'searchparameters.md'
#path = pathlib.Path(r'/Users/ehaas/Documents/FHIR/US-Core-R4/source/pages/_includes')
path.write_text(searchparameters_md)

!Patient True
!Patient True


---
title: Operations and Search Parameters
layout: default
topofpage: true
sectionnumbering: true
---

The following operations and search parameters have been defined for the US Core Implementation Guide.  For more information on the [FHIR RESTful operations], [FHIR RESTful search api] and the standard [Search Param Registry] see the FHIR specification.

### Operations


  { % include list-simple-operationdefinitions.xhtml % }


<br />

### Search Parameters


**AllergyIntolerance**
  - [_id](SearchParameter-None.html)

**Encounter**
  - [_id](SearchParameter-None.html)
  - [patient](SearchParameter-us-core-encounter-patient.html)
  - [date](SearchParameter-us-core-encounter-date.html)
  - [identifier](SearchParameter-us-core-encounter-identifier.html)
  - [status](SearchParameter-us-core-encounter-status.html)
  - [class](SearchParameter-us-core-encounter-class.html)
  - [type](SearchParameter-us-core-encounter-type.html)

**Questionnaire**
  - [_id](SearchParameter-None.html)
  - [url](SearchParameter-us-core-questionnaire-url.html)
  - [status](SearchParameter-us-core-questionnaire-status.html)
  - [title](SearchParameter-us-core-questionnaire-title.html)
  - [publisher](SearchParameter-us-core-questionnaire-publisher.html)
  - [version](SearchParameter-us-core-questionnaire-version.html)
  - [context-type-value](SearchParameter-us-core-questionnaire-context-type-value.html)

**Patient**
  - [_id](SearchParameter-None.html)
  - [identifier](SearchParameter-us-core-patient-identifier.html)
  - [name](SearchParameter-us-core-patient-name.html)
  - [birthdate](SearchParameter-us-core-patient-birthdate.html)
  - [gender](SearchParameter-us-core-patient-gender.html)
  - [family](SearchParameter-us-core-patient-family.html)
  - [given](SearchParameter-us-core-patient-given.html)


{% include link_list.md %}

1837